In [ ]:
# @title Setup

competition = "thai-dialect-classification-central-northern"  # @param
# @markdown ---

from google.colab import userdata
import json

# Get the Kaggle credentials from Colab's userdata
username = userdata.get("KAGGLE_USER")
key = userdata.get("KAGGLE_KEY")

# Echo the credentials into the kaggle.json file
!mkdir -p ~/.kaggle
!echo '{{"username":"{username}","key":"{key}"}}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

competition_name = "thai-dialect-classification-central-northern"
dir_name = "north-central-classification"

! kaggle competitions download -c {competition_name}
! mkdir {dir_name}
! unzip /content/{competition_name}.zip -d {dir_name}

thai-dialect-classification-central-northern.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘north-central-classification’: File exists
Archive:  /content/thai-dialect-classification-central-northern.zip
replace north-central-classification/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!pip install -q pythainlp
!pip install pycaret[full]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of category-encoders to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.9/109.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 41.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of flask to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━

## Overview

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from pythainlp.corpus.common import thai_stopwords

In [ ]:
df = pd.read_csv('/content/north-central-classification/train.csv', index_col='id')
df.head()

,Sentence,Class
id,,
0,ตึ้ง บ่า มี เวียก มี ก๋าน หยัง เยียะ กั๋น เน้า...,Northern
1,ของ ชิ้น นี้ ฉัน เป็น คน ซื้อ มา เธอ จะ มา ทึก...,Central
2,เป๋น แม่ ญิง หื้อ หมั่น หลับ เดิ้ก ลุก เจ๊า หื...,Northern
3,เมื่อ คืน วาน ฝน ตก หนัก พา ยุ เข้า ลม ไป พัด ...,Central
4,น้ำ บ่อ บ้าน เปิ้น เลิ้ก เก้า ข้อ ปอ เถิง หน้า...,Northern


In [ ]:
def preprocess_text(sentence):
    tokens = sentence.split()
    tokens = [word for word in tokens if word not in thai_stopwords()]  # Remove Stopwords
    return " ".join(tokens)

df_prep = df.copy()
# df_prep['Sentence'] = df_prep['Sentence'].apply(preprocess_text)

class_map = {'Northern': 0, 'Central': 1}
df_prep['Class'] = df_prep['Class'].map(class_map)
df_prep.head()

,Sentence,Class
id,,
0,ตึ้ง บ่า มี เวียก มี ก๋าน หยัง เยียะ กั๋น เน้า...,0
1,ของ ชิ้น นี้ ฉัน เป็น คน ซื้อ มา เธอ จะ มา ทึก...,1
2,เป๋น แม่ ญิง หื้อ หมั่น หลับ เดิ้ก ลุก เจ๊า หื...,0
3,เมื่อ คืน วาน ฝน ตก หนัก พา ยุ เข้า ลม ไป พัด ...,1
4,น้ำ บ่อ บ้าน เปิ้น เลิ้ก เก้า ข้อ ปอ เถิง หน้า...,0


In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,5), analyzer="char_wb", max_features=7000)

X_train = vectorizer.fit_transform(df_prep["Sentence"])
y_train = df["Class"]
X_train

<2000x7000 sparse matrix of type '<class 'numpy.float64'>'
	with 463167 stored elements in Compressed Sparse Row format>

## Model Selection

In [ ]:
from pycaret.classification import *
import torch

In [ ]:
clf_setup = setup(data=X_train, target=y_train, session_id=42,
                  preprocess=False, train_size=0.9,
                  use_gpu=torch.cuda.is_available())
best_model = compare_models(sort='Accuracy', include=['nb', 'lr', 'svm', 'rf', 'xgboost',])
best_model = create_model(best_model)

,Description,Value
0,Session id,42
1,Target,Class
2,Target type,Binary
3,Original data shape,"(2000, 7001)"
4,Transformed data shape,"(2000, 7001)"
5,Transformed train set shape,"(1800, 7001)"
6,Transformed test set shape,"(200, 7001)"
7,Numeric features,7000


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9978,1.0000,0.9978,0.9978,0.9978,0.9956,0.9956,5.3970
svm,SVM - Linear Kernel,0.9961,1.0000,0.9961,0.9962,0.9961,0.9922,0.9923,5.2320
rf,Random Forest Classifier,0.9933,0.9999,0.9933,0.9934,0.9933,0.9867,0.9867,6.1430


Processing:   0%|          | 0/25 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9944,1.0000,0.9944,0.9945,0.9944,0.9889,0.9889
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,0.9944,1.0000,0.9944,0.9945,0.9944,0.9889,0.9889
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,0.9889,0.9998,0.9889,0.9889,0.9889,0.9778,0.9778


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
evaluate_model(best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

## Predictions

In [ ]:
test_df = pd.read_csv('/content/north-central-classification/test.csv', index_col='id')
# test_df['Sentence'] = test_df['Sentence'].apply(preprocess_text)
test_df.head()

,Sentence
id,
0,บอก น้อง ว่า ค่อย ค่อย เดิน อย่า รีบ ร้อย เกิด...
1,ไอ้ จ่อน นอน สะ ดุ้ง ตื่น นึก ว่า โดน ผี อำ ที...
2,มอง ไป ทาง ไหน ก็ มี แต่ อา หาร อร่อย ตึง นั้น...
3,ห้าม ไป แอ่ว มะ ค่ำ มะ คืน เน้อ แม่ หัน ข่าว ล...
4,คน ล้าน นา เขา เชื่อ ว่า พระ พุทธเจ้า ยัง มี ช...


In [ ]:
X_test = vectorizer.transform(test_df["Sentence"])
y_pred = best_model.predict(X_test)
len(y_pred)

2000

In [ ]:
submission = pd.read_csv('/content/north-central-classification/sample_submission.csv')
map_back = dict(zip(class_map.values(), class_map.keys()))

submission.loc[4:, 'Class'] = pd.Series(y_pred)[4:]
submission.head()

,id,Class
0,0,Central
1,1,Central
2,2,Central
3,3,Northern
4,4,Central


Compare prev. with current submission

In [ ]:
legacy = pd.read_csv("rf_3.csv")
legacy.rename(columns={'Class': 'Previous Submission'}, inplace=True)
legacy['New Submission'] = submission['Class']
legacy['text'] = test_df['Sentence']

pd.set_option('display.max_colwidth', None)
legacy[legacy['Previous Submission'] != legacy['New Submission']]

,id,Previous Submission,New Submission,text
4,4,Northern,Central,คน ล้าน นา เขา เชื่อ ว่า พระ พุทธเจ้า ยัง มี ชีวิต อยู่ เพราะ ฉะ นั้น เขา ก็ เลย มี ประเพณี ทาน ข้าว ใหม่ ผิง ไฟ พระ เจ้า เขา กลัว ท่าน จะ หนาว
6,6,Northern,Central,วัน พรุ่ง นี้ แม่ จะ ไป เที่ยว ลำ พูน ใคร จะ ไป ก็ ตื่น เช้า นะ เดี๋ยว ไป สาย รถ จะ เยอะ
7,7,Northern,Central,ดู ผู้ ชาย คน นั้น สิ เขา กิน อา หาร อร่อย นะ สั่ง ข้าว มา กิน ครั้ง ละ สอง จาน ทุก วัน นะ
8,8,Northern,Central,วัน สิ้น ปี นี้ แม่ ว่า จะ ไป ซื้อ ของ ที่ ห้าง เอา ของ ไป ฝาก น้า ภัทร พี่ ของ พ่อ สัก หน่อย
12,12,Northern,Central,เมื่อ วาน ผู้ ใหญ่ บ้าน เขา นัด ประ ชุม กลุ่ม ชาว บ้าน ที่ วัด เห็น ว่า จะ สร้าง ถนน เข้า หมู่ บ้าน
...,...,...,...,...
1990,1990,Northern,Central,เธอ ซื้อ แกง อะ ไร มา เนี่ย ไม่ อร่อย ซัก นิด สู้ ลาบ ปลา ดุก ที่ พี่ ทำ ก็ ไม่ ได้
1994,1994,Northern,Central,ถนน คน เดิน หยุด มา หลาย อาทิตย์ แล้ว นะ แต่ อาทิตย์ หน้า จะ กลับ มา เปิด เหมือน เดิม แล้ว
1995,1995,Central,Northern,ไป สอบ ไป ขับ ขี่ มา เป๋น จะ ใด พ่อง ยาก ก่อ ปี้ สอบ เป๋น สาม สี่ รอบ ก่อน กว่า จะ ได้
1997,1997,Central,Northern,หัน ตุ๊ หลวง ว่า มี โยม จา กระ ยอง มา เป็น เจ้า ภาพ แป๋ง ก่ำ แปง วัด เฮา และ เน้อ


In [ ]:
submission.to_csv('lr_raw.csv', index=False)